<a href="https://colab.research.google.com/github/hugoluiz0607/ImersaoIA/blob/main/PlacaDoCarro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**OBJETIVO DO PROJETO**

Localizar veícluos e verificar se são roubados, clonados, ou estão com documentação atrasada.

-----

Changelog

Versão Alpha 0.1 - Funcionalidades

*   Analise de imagem para identificar veículo e placa.
*   Consulta da placa encontrada em API externa e validação de características físicas.
*   Validação se o mesmo veículo já foi localizado em outra oportunidade e se nesse intervalo de tempo foi possível que se deslocasse entre os dois locais.

-----

Implementações futuras - Funcionalidades
*   Multiplas análises em paralalelo.
*   Aplicativo mobile para rodar em background coletando fotos com autorização do usuário durante uma viagem. A coleta poderá ser recompensada, sendo ideal para motoristas de aplicativo que já utilizam o celuar e rodam durante todo dia.
*   Integração com bases de dados governamentais para integração de dados sobre roubos / situção do veículo.
*   Notificação automática de autoridades em caso de veículo roubado detectado.
*   Notificação educativa automática ao cidadão que estiver circulando sem estar em dia com pagamento dos impostos.
*   Notificação automática para proprietários de veículos que são possíveis vítimas de clone de veiculo.


In [67]:
!pip install -q -U google-generativeai

In [69]:
# Import the Python SDK
import google.generativeai as genai
import hashlib
import requests as req
import json

from pathlib import Path
from PIL import Image
from google.colab import userdata

#removido por segurança
#token = userdata.get('token_placas')
url = "https://wdapi2.com.br/consulta/"
GOOGLE_API_KEY = userdata.get('api_google')
genai.configure(api_key=GOOGLE_API_KEY)


# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

system_instruction = "aja como um especialista em veículos.\n seja objetivo e direto.\n só exiba informações sobre os carros que conseguir identificar a placa. não justifique as respostas."

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

chat = model.start_chat(history=[])


In [70]:
# Simulando posição da foto atual - Será capturado do GPS do celular que enviou a foto
posicaoGpsFoto = "-19.916039, -43.918500";
dataDaFotoAtual = "10/05/2024 09:25"

# Encontrando a cidade da posição GPS
promptCidade = "Aja como especialista em localização e responda com muita objetvidade: Qual é o nome da cidade e estado, com 2 digitos, da posição de gps " + posicaoGpsFoto;
cidadePosicaoGpsFotoAtual = chat.send_message(promptCidade).text

print(cidadePosicaoGpsFotoAtual)

# Simulando que o veículo já foi localizado em outra oportuniadde
cidadePosicaoGpsFotoAntiga = "Recife, PE"
dataDaFotoAntiga = "05/05/2024 09:25"

Belo Horizonte, MG 



In [72]:
# Localizando e analisando apenas 1 veículo por vez em cada foto

from PIL import Image
from io import BytesIO

# Imagem de teste
urlFoto = "https://www.ceara.gov.br/wp-content/uploads/2020/03/Placa-Mercosul-3.jpg"
response = req.get(urlFoto)
img = Image.open(BytesIO(response.content))

# Suba uma imagem com o nome "imagem_analise.jpg" que contenha um veículo com sua placa visivel.
#img = Image.open('/content/imagem_analise.jpg')
#img = Image.open()

prompt1 = "Retorne apenas a placa de 1 vículo, escolha o veículo que está mais proximo e com a placa mais legivel, sem espacos."

placa = chat.send_message(content=[img,prompt1], stream=True)
placa.resolve()
print(placa.text)

POX4G21



In [73]:
# Acessando API terceira para obter dados do veículo da placa encontrada na imagem.

# URL de produção ( poucos créditos disponíveis cedidos pela plataforma para desenvolvimento )
# Token removido por segurança
#urlConsulta = url + placa.text.replace('\n', '') + "/" + token

# URL fake para ativar o Mock
urlConsulta = url + placa.text.replace('\n', '') + "/tokenFake"
print(urlConsulta)

resposta = req.get(urlConsulta)
data = ""
print(resposta.text)

https://wdapi2.com.br/consulta/POX4G21/tokenFake
{"message": "Token Invalido "}



In [74]:
#Obtem resposta da chamada externa, caso retorne erro, utilize o Mock
if resposta.status_code != 200 :
  print(resposta.json()["message"])
  jsonResultado = {"MARCA":"FIAT","MODELO":"SIENA FIRE FLEX","SUBMODELO":"SIENA","VERSAO":"FLEX","ano":"2009","anoModelo":"2010","chassi":"*********","chassis":"*********","city":"Contagem","codigoRetorno":"0","codigoSituacao":"0","color":"PRATA","cor":"PRATA","data":"11/05/2024 10:06:51","dataAtualizacaoAlarme":"11/05/2024","dataAtualizacaoCaracteristicasVeiculo":"11/05/2024","dataAtualizacaoRouboFurto":"11/05/2024","datacache":"11/05/2024 10:06:51","date":"2024-05-11T10:06:51.283-03:00"}
else:
  jsonResultado = resposta.json()

#Obter apenas dados úteis para verificação
data = [jsonResultado["MARCA"], jsonResultado["MODELO"], jsonResultado["SUBMODELO"] ,jsonResultado["VERSAO"], jsonResultado["ano"], jsonResultado["cor"], jsonResultado["city"]]
print(data)

Token Invalido 
['FIAT', 'SIENA FIRE FLEX', 'SIENA', 'FLEX', '2009', 'PRATA', 'Contagem']


In [75]:
# Verificar se o veículo já foi fotografado outra vez e validar se foi possível realizar uma viagem do local anterior até o local atual.
# Se a diferenca entre as fotos menor que o período de uma viagem entre as cidades, a probabilidade de ser um veiculo clonado é alta.

promptTempoViagem = "Responda da seguinte forma: 'Tempo de viagem: ', 'Diferença entre datas: ', 'Viagem possível: '  Seja extremamente objetivo, responda em minutos. Sabendo qual é o tempo de viagem médio de carro entre " + cidadePosicaoGpsFotoAntiga + " para a cidade " + cidadePosicaoGpsFotoAtual + "?"
promptDiferencaEntreDatas = "Qual é iferenca em minutos entre as datas " + dataDaFotoAntiga + " e " + dataDaFotoAtual + "?"
resultadoViagemPossivel =  "Responda Sim ou Não. É possivel viajar entre as cidades no tempo da diferença entre as datas?"

resultadoTempoViagem = chat.send_message([promptTempoViagem,promptDiferencaEntreDatas,resultadoViagemPossivel])
print(resultadoTempoViagem.text)


Tempo de viagem: 1440 
Diferença entre datas: 7200
Viagem possível: Sim 



In [76]:
# Resultado final, apresentando a comparação o veículo encontrado na foto, com a descrção retornada pela API extrna
# e validando se o veículo teria condições de viajar entre as cidades. Utilizando esse critério como muito importante na validação

if data != "" :
  prompt2 = json.dumps(data)
  prompt3 = "Localize o carro com a placa " + placa.text + " e compare com os dados informados, analise se seria possivel que o carro estivesse na cidade " + cidadePosicaoGpsFotoAtual + " vindo de " + cidadePosicaoGpsFotoAntiga + ", utilize esse dado como crucial na sua probabilidade. Responda da seguinte forma:\n\n\nCarro da Imagem:\nMarca: \nModelo: \nRange de ano: \nCor: \nCidade: " + cidadePosicaoGpsFotoAtual +"\nPossibilidade de ter viajado de " + cidadePosicaoGpsFotoAntiga +  " a tempo de chegar em " + cidadePosicaoGpsFotoAtual + ": \nProbabilidade de ser o mesmo carro da descrição em %: \n\n"

  resultado = chat.send_message(content=[prompt2,prompt3])
  print(resultado.text)

Carro da Imagem:
Marca: Volkswagen
Modelo: Voyage
Range de ano: 2016-2018
Cor: Prata
Cidade: Belo Horizonte, MG 

Possibilidade de ter viajado de Recife, PE a tempo de chegar em Belo Horizonte, MG: Sim
Probabilidade de ser o mesmo carro da descrição em %: 0% 

